In [11]:
# Cloner YOLOv5 depuis GitHub et installer les dépendances
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17075, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 17075 (delta 19), reused 7 (delta 7), pack-reused 17049 (from 2)
Receiving objects: 100% (17075/17075), 15.68 MiB | 9.48 MiB/s, done.
Resolving deltas: 100% (11719/11719), done.
/content/yolov5/yolov5/yolov5


In [12]:
from google.colab import files

# Télécharger ta base de données (uploader le fichier)
uploaded = files.upload()

# Décompresser le fichier ZIP
import zipfile
import os

zip_path = 'Waste detection.v1i.createml.zip'  # Nom du fichier téléchargé
output_path = '/content/dataset'
os.makedirs(output_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_path)

print("Décompression terminée !")


Saving Waste detection.v1i.createml.zip to Waste detection.v1i.createml.zip
Décompression terminée !


In [13]:
import shutil

# Organiser uniquement les images pour YOLOv5
images_path = '/content/dataset/train'  # Chemin des images extraites

# Créer les dossiers nécessaires pour YOLOv5
!mkdir -p /content/dataset/yolo/train/images
!mkdir -p /content/dataset/yolo/train/labels

# Déplacer uniquement les fichiers images
for file in os.listdir(images_path):
    if file.endswith('.jpg'):
        shutil.move(os.path.join(images_path, file), '/content/dataset/yolo/train/images')

print("Les images ont été déplacées avec succès !")



Les images ont été déplacées avec succès !


In [14]:
# Créer un fichier de configuration YAML pour YOLOv5
yaml_content = """
path: /content/dataset/yolo
train: train/images
val: train/images

# Classes
nc: 4
names: ['plastic', 'cardboard', 'glass', 'metal']
"""

with open("waste.yaml", "w") as f:
    f.write(yaml_content)

print("Fichier waste.yaml créé avec succès !")


Fichier waste.yaml créé avec succès !


In [15]:
# Lancer l'entraînement YOLOv5
!python train.py --img 640 --batch 16 --epochs 50 --data waste.yaml --weights yolov5s.pt


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-18 03:23:41.314569: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-18 03:23:41.375715: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-18 03:23:41.392572: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visua

In [16]:
import os

labels_path = '/content/dataset/yolo/train/labels'

# Lister les fichiers .txt dans le dossier des labels
label_files = [f for f in os.listdir(labels_path) if f.endswith('.txt')]
print(f"Nombre de fichiers labels : {len(label_files)}")
print("Exemples de fichiers :", label_files[:5])


Nombre de fichiers labels : 0
Exemples de fichiers : []


In [17]:
# Charger et afficher un aperçu des annotations CreateML
annotations_path = '/content/dataset/train/_annotations.createml.json'

import json

with open(annotations_path, 'r') as f:
    data = json.load(f)

# Afficher les 2 premières annotations pour comprendre la structure
print("Aperçu des annotations :", data[:2])


Aperçu des annotations : [{'image': 'IMG_9152_39_11zon_jpg.rf.ea6250c449175831fd1b91f7d3ab2cfe.jpg', 'annotations': [{'label': 'plastic', 'coordinates': {'x': 206.5, 'y': 196.5, 'width': 168.5, 'height': 108.5}}, {'label': 'plastic', 'coordinates': {'x': 435, 'y': 213, 'width': 159, 'height': 124.5}}]}, {'image': 'IMG_9168_53_11zon_jpg.rf.e9d3b557fc2818ee5d76db2df02e0f51.jpg', 'annotations': [{'label': 'cardboard', 'coordinates': {'x': 308, 'y': 336.5, 'width': 341, 'height': 296}}]}]


In [18]:
import cv2
import os

images_dir = '/content/dataset/yolo/train/images'
image_path = os.path.join(images_dir, 'IMG_9152_39_11zon_jpg.rf.ea6250c449175831fd1b91f7d3ab2cfe.jpg')

# Lire l'image pour obtenir ses dimensions
image = cv2.imread(image_path)
height, width, _ = image.shape
print(f"Dimensions de l'image : largeur={width}, hauteur={height}")


Dimensions de l'image : largeur=640, hauteur=640


In [19]:
# Chemins
output_labels_path = '/content/dataset/yolo/train/labels'
os.makedirs(output_labels_path, exist_ok=True)

# Classes
classes = ['plastic', 'cardboard', 'glass', 'metal']

# Dimensions fixes de l'image
image_width = 640
image_height = 640

# Convertir les annotations CreateML vers le format YOLO
for item in data:
    image_name = item['image']
    label_file = os.path.join(output_labels_path, image_name.replace('.jpg', '.txt'))

    with open(label_file, 'w') as f:
        for ann in item['annotations']:
            class_id = classes.index(ann['label'])  # ID de la classe
            x_center = ann['coordinates']['x'] / image_width
            y_center = ann['coordinates']['y'] / image_height
            width = ann['coordinates']['width'] / image_width
            height = ann['coordinates']['height'] / image_height

            # Écrire les données dans le fichier texte
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

print("✅ Conversion des annotations au format YOLO réussie !")


✅ Conversion des annotations au format YOLO réussie !


In [20]:
# Afficher les fichiers labels générés
label_files = os.listdir(output_labels_path)
print(f"Nombre de fichiers labels générés : {len(label_files)}")
print("Exemples de fichiers :", label_files[:5])

# Afficher le contenu d'un fichier label pour vérifier
with open(os.path.join(output_labels_path, label_files[0]), 'r') as f:
    print("\nContenu du fichier label :", f.read())


Nombre de fichiers labels générés : 519
Exemples de fichiers : ['IMG_9208_82_11zon_jpg.rf.a71932958971d347d65148507ea319bb.txt', 'IMG_9112_10_11zon_jpg.rf.819f62dea08cb7c33f2dca72d0fefbba.txt', 'IMG_9110_8_11zon_jpg.rf.62c7a2072c197d3dd17f4fe504cb7ae9.txt', 'IMG_9270_10_11zon_jpg.rf.d2815a82bed49af7d81b2824670edcda.txt', 'Image_1_11zon_jpeg.rf.69d2ed02752fdc95fdcb3a61d3219863.txt']

Contenu du fichier label : 2 0.375000 0.510938 0.673438 0.553125



In [21]:
!WANDB_MODE=disabled python train.py --img 640 --batch 16 --epochs 50 --data waste.yaml --weights yolov5s.pt



wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-18 03:26:32.387034: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-18 03:26:32.419917: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-18 03:26:32.429640: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=waste.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=

In [22]:
from google.colab import drive
drive.mount('/content/drive')

# Copier les fichiers de poids dans Google Drive
!cp runs/train/exp/weights/last.pt /content/drive/MyDrive/
!cp runs/train/exp/weights/best.pt /content/drive/MyDrive/


Mounted at /content/drive
cp: cannot stat 'runs/train/exp/weights/last.pt': No such file or directory
cp: cannot stat 'runs/train/exp/weights/best.pt': No such file or directory


In [ ]:
!python train.py --resume


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-18 09:10:48.155135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-18 09:10:48.190455: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-18 09:10:48.201183: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visua

In [15]:
import os
os.environ['WANDB_MODE'] = 'disabled'
print("Weights & Biases (W&B) désactivé !")


Weights & Biases (W&B) désactivé !


In [16]:
!python train.py --img 640 --batch 16 --epochs 50 --data waste.yaml --weights yolov5s.pt


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-17 14:23:48.904262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-17 14:23:48.933764: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-17 14:23:48.941900: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=waste.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=

In [17]:
# Supprimer complètement l'intégration avec W&B de YOLOv5
!sed -i 's|wandb.init.*|pass|' train.py
!sed -i 's|wandb.log.*|pass|' train.py
!sed -i 's|wandb.finish.*|pass|' train.py

print("Weights & Biases (W&B) désactivé du code source !")


Weights & Biases (W&B) désactivé du code source !


In [19]:
!python train.py --img 640 --batch 16 --epochs 50 --data waste.yaml --weights yolov5s.pt


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-17 14:48:27.802563: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-17 14:48:27.840492: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-17 14:48:27.851457: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=waste.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=

In [4]:
!python train.py --resume


python3: can't open file '/content/train.py': [Errno 2] No such file or directory


In [3]:
!python train.py --resume


python3: can't open file '/content/train.py': [Errno 2] No such file or directory


In [5]:
# Re-télécharger le dépôt YOLOv5
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17075, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 17075 (delta 19), reused 7 (delta 7), pack-reused 17049 (from 2)
Receiving objects: 100% (17075/17075), 15.68 MiB | 10.38 MiB/s, done.
Resolving deltas: 100% (11719/11719), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.3/901.3 kB 14.9 MB/s eta 0:00:00


In [6]:
import os
if os.path.exists('runs/train/exp5/weights/last.pt'):
    print("✅ Le fichier 'last.pt' existe. Prêt à reprendre l'entraînement.")
else:
    print("❌ Le fichier 'last.pt' est manquant. L'entraînement devra recommencer.")


❌ Le fichier 'last.pt' est manquant. L'entraînement devra recommencer.


In [7]:
# Cloner le dépôt YOLOv5
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
# Installer les dépendances
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17075, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 17075 (delta 19), reused 7 (delta 7), pack-reused 17049 (from 2)
Receiving objects: 100% (17075/17075), 15.69 MiB | 18.61 MiB/s, done.
Resolving deltas: 100% (11724/11724), done.
/content/yolov5/yolov5


In [8]:
!python train.py --img 640 --batch 16 --epochs 50 --data waste.yaml --weights yolov5s.pt


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-18 03:11:18.360357: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-18 03:11:18.384527: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-18 03:11:18.391891: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to regi

In [9]:
import os
if os.path.exists('waste.yaml'):
    print("✅ Le fichier 'waste.yaml' existe.")
else:
    print("❌ Le fichier 'waste.yaml' est introuvable.")


❌ Le fichier 'waste.yaml' est introuvable.


In [10]:
train: /content/dataset/yolo/train/images
val: /content/dataset/yolo/val/images

nc: 4  # Nombre de classes (par exemple : plastique, carton, verre, métal)
names: ['plastic', 'cardboard', 'glass', 'metal']  # Noms des classes


SyntaxError: invalid syntax (<ipython-input-10-00467df304e0>, line 1)